<a href="https://colab.research.google.com/github/sumukshashidhar/toreda/blob/master/src/sentiment%20analysis/Twitter_Stream_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
!pip install newspaper3k
!pip install vaderSentiment

In [91]:
import sys
import json
import time
import re
import requests
import nltk
import argparse
import logging
import string
try:
    import urllib.parse as urlparse
except ImportError:
    import urlparse
from tweepy.streaming import StreamListener
from tweepy import API, Stream, OAuthHandler, TweepError
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
from random import randint, randrange
from datetime import datetime
from newspaper import Article, ArticleException

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
twitter_users_file = './twitteruserids.txt'
tweet_ids = []
sentimentURL = 'http://text-processing.com/api/sentiment/'


In [0]:
def get_twitter_users_from_file(file):
    # get twitter user ids from text file
    twitter_users = []
    try:
        f = open(file, "rt", encoding='utf-8')
        for line in f.readlines():
            u = line.strip()
            twitter_users.append(u)
        f.close()
    except (IOError, OSError) as e:
        pass
    return twitter_users

In [0]:
twitter_users = get_twitter_users_from_file(twitter_users_file)
if len(twitter_users) > 0:
    useridlist = twitter_users
else:
    sys.exit(1)

In [0]:
consumer_key = "FUPWR8oiyCRa7RwLY5mqSQdR0"
consumer_secret = "U7qijEyqCegOr4gdwRW03X9ypTP72biZgFLJW1qzyXiVitQEQr"
access_token = "762166371765456897-6lLHVr7GyrvYYCxGDWIelZFLThvnoqj"
access_token_secret = "RPwSGoJNDJh7fsCKlMum1ErMoLmLUj9jHkDBRKciYcBsS"
nltk_tokens_required = ("neuralink", "solar", "tesla", "@tesla", "#tesla", "tesla", "tsla", "#tsla", "elonmusk", "elon", "musk", "spacex", "starlink")
nltk_min_tokens = 1
nltk_tokens_ignored = ("win", "giveaway")
twitter_feeds = ["@elonmusk", "@cnbc", "@benzinga", "@stockwits",
                 "@Newsweek", "@WashingtonPost", "@breakoutstocks", "@bespokeinvest",
                 "@WSJMarkets", "@stephanie_link", "@nytimesbusiness", "@IBDinvestors",
                 "@WSJDealJournal", "@jimcramer", "@TheStalwart", "@TruthGundlach",
                 "@Carl_C_Icahn", "@ReformedBroker", "@bespokeinvest", "@stlouisfed",
                 "@muddywatersre", "@mcuban", "@AswathDamodaran", "@elerianm",
                 "@MorganStanley", "@ianbremmer", "@GoldmanSachs", "@Wu_Tang_Finance",
                 "@Schuldensuehner", "@NorthmanTrader", "@Frances_Coppola", "@bySamRo",
                 "@BuzzFeed","@nytimes"]

In [0]:

def clean_text(text):
    # clean up text
    text = text.replace("\n", " ")
    text = re.sub(r"https?\S+", "", text)
    text = re.sub(r"&.*?;", "", text)
    text = re.sub(r"<.*?>", "", text)
    text = text.replace("RT", "")
    text = text.replace(u"…", "")
    text = text.strip()
    return text


def clean_text_sentiment(text):
    # clean up text for sentiment analysis
    text = re.sub(r"[#|@]\S+", "", text)
    text = text.strip()
    return text


def get_sentiment_from_url(text, sentimentURL):
    # get sentiment from text processing website
    payload = {'text': text}

    try:
        #logger.debug(text)
        post = requests.post(sentimentURL, data=payload)
        #logger.debug(post.status_code)
        #logger.debug(post.text)
    except requests.exceptions.RequestException as re:
        logger.error("Exception: requests exception getting sentiment from url caused by %s" % re)
        raise

    # return None if we are getting throttled or other connection problem
    if post.status_code != 200:
        logger.warning("Can't get sentiment from url caused by %s %s" % (post.status_code, post.text))
        return None

    response = post.json()

    neg = response['probability']['neg']
    pos = response['probability']['pos']
    neu = response['probability']['neutral']
    label = response['label']

    # determine if sentiment is positive, negative, or neutral
    if label == "neg":
        sentiment = "negative"
    elif label == "neutral":
        sentiment = "neutral"
    else:
        sentiment = "positive"

    return sentiment, neg, pos, neu


def sentiment_analysis(text):
    """Determine if sentiment is positive, negative, or neutral
    algorithm to figure out if sentiment is positive, negative or neutral
    uses sentiment polarity from TextBlob, VADER Sentiment and
    sentiment from text-processing URL
    could be made better :)
    Uploads sentiment to stocksight website.
    """

    # pass text into sentiment url
    if True:
        ret = get_sentiment_from_url(text, sentimentURL)
        if ret is None:
            sentiment_url = None
        else:
            sentiment_url, neg_url, pos_url, neu_url = ret
    else:
        sentiment_url = None

    # pass text into TextBlob
    text_tb = TextBlob(text)

    # pass text into VADER Sentiment
    analyzer = SentimentIntensityAnalyzer()
    text_vs = analyzer.polarity_scores(text)

    # determine sentiment from our sources
    if sentiment_url is None:
        if text_tb.sentiment.polarity < 0 and text_vs['compound'] <= -0.05:
            sentiment = "negative"
        elif text_tb.sentiment.polarity > 0 and text_vs['compound'] >= 0.05:
            sentiment = "positive"
        else:
            sentiment = "neutral"
    else:
        if text_tb.sentiment.polarity < 0 and text_vs['compound'] <= -0.05 and sentiment_url == "negative":
            sentiment = "negative"
        elif text_tb.sentiment.polarity > 0 and text_vs['compound'] >= 0.05 and sentiment_url == "positive":
            sentiment = "positive"
        else:
            sentiment = "neutral"

    # calculate average polarity from TextBlob and VADER
    polarity = (text_tb.sentiment.polarity + text_vs['compound']) / 2

    # output sentiment polarity
    print("************")
    print("Sentiment Polarity: " + str(round(polarity, 3)))

    # output sentiment subjectivity (TextBlob)
    print("Sentiment Subjectivity: " + str(round(text_tb.sentiment.subjectivity, 3)))

    # output sentiment
    print("Sentiment (url): " + str(sentiment_url))
    print("Sentiment (algorithm): " + str(sentiment))
    print("Overall sentiment (textblob): ", text_tb.sentiment) 
    print("Overall sentiment (vader): ", text_vs) 
    print("sentence was rated as ", round(text_vs['neg']*100, 3), "% Negative") 
    print("sentence was rated as ", round(text_vs['neu']*100, 3), "% Neutral") 
    print("sentence was rated as ", round(text_vs['pos']*100, 3), "% Positive") 
    print("************")


    #TODO: add all this to a list, and make an animated graph!
    return polarity, text_tb.sentiment.subjectivity, sentiment


In [0]:
class TweetStreamListener(StreamListener):

    def __init__(self):
        self.count = 0
        self.count_filtered = 0
        self.filter_ratio = 0

    # on success
    def on_data(self, data):
        try:
            self.count+=1
            # decode json
            dict_data = json.loads(data)

            print("\n------------------------------> (tweets: %s, filtered: %s, filter-ratio: %s)" \
                % (self.count, self.count_filtered, str(round(self.count_filtered/self.count*100,2))+"%"))
            logger.debug('tweet data: ' + str(dict_data))

            text = dict_data["text"]
            if text is None:
                logger.info("Tweet has no relevant text, skipping")
                self.count_filtered+=1
                return True

            # grab html links from tweet
            tweet_urls = []
            if True:
                tweet_urls = re.findall(r'(https?://[^\s]+)', text)

            # clean up tweet text
            textclean = clean_text(text)

            # check if tweet has no valid text
            if textclean == "":
                logger.info("Tweet does not cotain any valid text after cleaning, not adding")
                self.count_filtered+=1
                return True

            # get date when tweet was created
            created_date = time.strftime(
                '%Y-%m-%dT%H:%M:%S', time.strptime(dict_data['created_at'], '%a %b %d %H:%M:%S +0000 %Y'))

            # store dict_data into vars
            screen_name = str(dict_data.get("user", {}).get("screen_name"))
            location = str(dict_data.get("user", {}).get("location"))
            language = str(dict_data.get("user", {}).get("lang"))
            friends = int(dict_data.get("user", {}).get("friends_count"))
            followers = int(dict_data.get("user", {}).get("followers_count"))
            statuses = int(dict_data.get("user", {}).get("statuses_count"))
            text_filtered = str(textclean)
            tweetid = int(dict_data.get("id"))
            text_raw = str(dict_data.get("text"))

            # output twitter data
            print("\n<------------------------------")
            print("Tweet Date: " + created_date)
            print("Screen Name: " + screen_name)
            print("Location: " + location)
            print("Language: " + language)
            print("Friends: " + str(friends))
            print("Followers: " + str(followers))
            print("Statuses: " + str(statuses))
            print("Tweet ID: " + str(tweetid))
            print("Tweet Raw Text: " + text_raw)
            print("Tweet Filtered Text: " + text_filtered)

            # create tokens of words in text using nltk
            text_for_tokens = re.sub(
                r"[\%|\$|\.|\,|\!|\:|\@]|\(|\)|\#|\+|(``)|('')|\?|\-", "", text_filtered)
            tokens = nltk.word_tokenize(text_for_tokens)
            # convert to lower case
            tokens = [w.lower() for w in tokens]
            # remove punctuation from each word
            table = str.maketrans('', '', string.punctuation)
            stripped = [w.translate(table) for w in tokens]
            # remove remaining tokens that are not alphabetic
            tokens = [w for w in stripped if w.isalpha()]
            # filter out stop words
            stop_words = set(nltk.corpus.stopwords.words('english'))
            tokens = [w for w in tokens if not w in stop_words]
            # remove words less than 3 characters
            tokens = [w for w in tokens if not len(w) < 3]
            print("NLTK Tokens: " + str(tokens))

            # check for min token length
            if len(tokens) < 5:
                logger.info("Tweet does not contain min. number of tokens, not adding")
                self.count_filtered+=1
                return True

            # do some checks before adding to elasticsearch and crawling urls in tweet
            if friends == 0 or \
                            followers == 0 or \
                            statuses == 0 or \
                            text == "" or \
                            tweetid in tweet_ids:
                logger.info("Tweet doesn't meet min requirements, not adding")
                self.count_filtered+=1
                return True

            # check ignored tokens from config
            for t in nltk_tokens_ignored:
                if t in tokens:
                    logger.info("Tweet contains token from ignore list, not adding")
                    self.count_filtered+=1
                    return True
            # check required tokens from config
            tokenspass = False
            tokensfound = 0
            for t in nltk_tokens_required:
                if t in tokens:
                    tokensfound += 1
                    if tokensfound == nltk_min_tokens:
                        tokenspass = True
                        break
            if not tokenspass:
                logger.info("Tweet does not contain token from required list or min required, not adding")
                self.count_filtered+=1
                return True

            # clean text for sentiment analysis
            text_clean = clean_text_sentiment(text_filtered)

            # check if tweet has no valid text
            if text_clean == "":
                logger.info("Tweet does not cotain any valid text after cleaning, not adding")
                self.count_filtered+=1
                return True

            print("Tweet Clean Text (sentiment): " + text_clean)

            # get sentiment values
            polarity, subjectivity, sentiment = sentiment_analysis(text_clean)

            # add tweet_id to list
            tweet_ids.append(dict_data["id"])

            # get sentiment for tweet 
            if len(tweet_urls) > 0:
                tweet_urls_polarity = 0
                tweet_urls_subjectivity = 0
                for url in tweet_urls:
                    res = tweeklink_sentiment_analysis(url)
                    if res is None:
                        continue
                    pol, sub, sen = res
                    tweet_urls_polarity = (tweet_urls_polarity + pol) / 2
                    tweet_urls_subjectivity = (tweet_urls_subjectivity + sub) / 2
                    if sentiment == "positive" or sen == "positive":
                        sentiment = "positive"
                    elif sentiment == "negative" or sen == "negative":
                        sentiment = "negative"
                    else:
                        sentiment = "neutral"

                # calculate average polarity and subjectivity from tweet and tweet links
                if tweet_urls_polarity > 0:
                    polarity = (polarity + tweet_urls_polarity) / 2
                if tweet_urls_subjectivity > 0:
                    subjectivity = (subjectivity + tweet_urls_subjectivity) / 2
            

            body={"author": screen_name,
                "location": location,
                "language": language,
                "friends": friends,
                "followers": followers,
                "statuses": statuses,
                "date": created_date,
                "message": text_filtered,
                "tweet_id": tweetid,
                "polarity": polarity,
                "subjectivity": subjectivity,
                "sentiment": sentiment}
            
            print(body)

            # randomly sleep to stagger request time
            time.sleep(randrange(2,5))
            return True

        except Exception as e:
            logger.warning("Exception: exception caused by: %s" % e)
            raise

    # on failure
    def on_error(self, status_code):
        logger.error("Got an error with status code: %s (will try again later)" % status_code)
        # randomly sleep to stagger request time
        time.sleep(randrange(2,30))
        return True

    # on timeout
    def on_timeout(self):
        logger.warning("Timeout... (will try again later)")
        # randomly sleep to stagger request time
        time.sleep(randrange(2,30))
        return True


In [0]:
tweetlistener = TweetStreamListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = API(auth)
stream = Stream(auth, tweetlistener)

In [0]:
logger = logging.getLogger('stocksight')
logger.setLevel(logging.INFO)
eslogger = logging.getLogger('elasticsearch')
eslogger.setLevel(logging.WARNING)
tweepylogger = logging.getLogger('tweepy')
tweepylogger.setLevel(logging.INFO)
requestslogger = logging.getLogger('requests')
requestslogger.setLevel(logging.INFO)
logging.addLevelName(
    logging.INFO, "\033[1;32m%s\033[1;0m"
                    % logging.getLevelName(logging.INFO))
logging.addLevelName(
    logging.WARNING, "\033[1;31m%s\033[1;0m"
                        % logging.getLevelName(logging.WARNING))
logging.addLevelName(
    logging.ERROR, "\033[1;41m%s\033[1;0m"
                    % logging.getLevelName(logging.ERROR))
logging.addLevelName(
    logging.DEBUG, "\033[1;33m%s\033[1;0m"
                    % logging.getLevelName(logging.DEBUG))
logformatter = '%(asctime)s [%(levelname)s][%(name)s] %(message)s'
loglevel = logging.INFO
logging.basicConfig(format=logformatter, level=loglevel)

In [100]:
logger.info('NLTK tokens required: ' + str(nltk_tokens_required))
logger.info('NLTK tokens ignored: ' + str(nltk_tokens_ignored))
logger.info('Listening for Tweets (ctrl-c to exit)...')
 
logger.info('No keywords entered, following Twitter users...')
logger.info('Twitter Feeds: ' + str(twitter_feeds))
logger.info('Twitter User Ids: ' + str(useridlist))
stream.filter(follow=useridlist, languages=['en'])


2020-05-07 04:47:56,903 [INFO][stocksight] NLTK tokens required: ('neuralink', 'solar', 'tesla', '@tesla', '#tesla', 'tesla', 'tsla', '#tsla', 'elonmusk', 'elon', 'musk', 'spacex', 'starlink')
2020-05-07 04:47:56,904 [INFO][stocksight] NLTK tokens ignored: ('win', 'giveaway')
2020-05-07 04:47:56,904 [INFO][stocksight] Listening for Tweets (ctrl-c to exit)...
2020-05-07 04:47:56,906 [INFO][stocksight] No keywords entered, following Twitter users...
2020-05-07 04:47:56,907 [INFO][stocksight] Twitter Feeds: ['@elonmusk', '@cnbc', '@benzinga', '@stockwits', '@Newsweek', '@WashingtonPost', '@breakoutstocks', '@bespokeinvest', '@WSJMarkets', '@stephanie_link', '@nytimesbusiness', '@IBDinvestors', '@WSJDealJournal', '@jimcramer', '@TheStalwart', '@TruthGundlach', '@Carl_C_Icahn', '@ReformedBroker', '@bespokeinvest', '@stlouisfed', '@muddywatersre', '@mcuban', '@AswathDamodaran', '@elerianm', '@MorganStanley', '@ianbremmer', '@GoldmanSachs', '@Wu_Tang_Finance', '@Schuldensuehner', '@NorthmanTr


------------------------------> (tweets: 1, filtered: 0, filter-ratio: 0.0%)

<------------------------------
Tweet Date: 2020-05-07T04:47:54
Screen Name: MatesonG
Location: Fort Worth, TX
Language: None
Friends: 80
Followers: 65
Statuses: 125
Tweet ID: 1258257224453021696
Tweet Raw Text: RT @mcuban: The One Thing in Life We Can Control is Our Effort
Tweet Filtered Text: @mcuban: The One Thing in Life We Can Control is Our Effort
NLTK Tokens: ['mcuban', 'one', 'thing', 'life', 'control', 'effort']


2020-05-07 04:48:00,260 [INFO][stocksight] Tweet does not contain token from required list or min required, not adding



------------------------------> (tweets: 2, filtered: 1, filter-ratio: 50.0%)

<------------------------------
Tweet Date: 2020-05-07T04:47:55
Screen Name: kibetterry
Location: Bomet,Kenya
Language: None
Friends: 1261
Followers: 1099
Statuses: 2469
Tweet ID: 1258257228282441728
Tweet Raw Text: RT @washingtonpost: Opinion: The "wartime president" has gone AWOL. More Americans will die. https://t.co/weOno4rwf9
Tweet Filtered Text: @washingtonpost: Opinion: The "wartime president" has gone AWOL. More Americans will die.
NLTK Tokens: ['washingtonpost', 'opinion', 'wartime', 'president', 'gone', 'awol', 'americans', 'die']

------------------------------> (tweets: 3, filtered: 2, filter-ratio: 66.67%)

<------------------------------
Tweet Date: 2020-05-07T04:47:55
Screen Name: JaylanBtg
Location: None
Language: None
Friends: 755
Followers: 1171
Statuses: 67955
Tweet ID: 1258257231109382145
Tweet Raw Text: RT @elonmusk: Wishing good vibes for all in 2nd half 2020
Tweet Filtered Text: @elon

2020-05-07 04:48:05,197 [INFO][stocksight] Tweet does not contain token from required list or min required, not adding


...

2020-05-07 04:49:03,331 [INFO][stocksight] Tweet does not contain token from required list or min required, not adding



------------------------------> (tweets: 67, filtered: 56, filter-ratio: 83.58%)

<------------------------------
Tweet Date: 2020-05-07T04:48:56
Screen Name: popopoikui
Location: Not in ur butthole
Language: None
Friends: 751
Followers: 2750
Statuses: 264
Tweet ID: 1258257483467911168
Tweet Raw Text: RT @washingtonpost: The urgent quest for a coronavirus treatment involves door-to-door blood collection and a llama named Winter https://t.…
Tweet Filtered Text: @washingtonpost: The urgent quest for a coronavirus treatment involves door-to-door blood collection and a llama named Winter
NLTK Tokens: ['washingtonpost', 'urgent', 'quest', 'coronavirus', 'treatment', 'involves', 'doortodoor', 'blood', 'collection', 'llama', 'named', 'winter']

------------------------------> (tweets: 68, filtered: 57, filter-ratio: 83.82%)

<------------------------------
Tweet Date: 2020-05-07T04:48:56
Screen Name: morgaliphoto
Location: Sammamish, WA
Language: None
Friends: 257
Followers: 43
Statuses: 425

2020-05-07 04:49:07,515 [INFO][stocksight] Tweet does not contain token from required list or min required, not adding
2020-05-07 04:49:07,696 [WARNING][stocksight] Exception: exception caused by: name 'tweeklink_sentiment_analysis' is not defined



------------------------------> (tweets: 69, filtered: 57, filter-ratio: 82.61%)

<------------------------------
Tweet Date: 2020-05-07T04:48:56
Screen Name: GATORPILOTBOB
Location: Atlanta, GA
Language: None
Friends: 187
Followers: 124
Statuses: 19698
Tweet ID: 1258257487234576384
Tweet Raw Text: RT @mcuban: The One Thing in Life We Can Control is Our Effort
Tweet Filtered Text: @mcuban: The One Thing in Life We Can Control is Our Effort
NLTK Tokens: ['mcuban', 'one', 'thing', 'life', 'control', 'effort']

------------------------------> (tweets: 70, filtered: 58, filter-ratio: 82.86%)

<------------------------------
Tweet Date: 2020-05-07T04:48:59
Screen Name: Uddand_balak
Location: Madhya Pradesh, India
Language: None
Friends: 21
Followers: 2
Statuses: 23
Tweet ID: 1258257499146407936
Tweet Raw Text: @elonmusk named his kid X AE A-12, meaning I have to find a different name for my child now. Why do all the good on… https://t.co/2pXdn4jMvh
Tweet Filtered Text: @elonmusk named his 

NameError: ignored